# Preprocessamento de Dados – Housing Califórnia

Preparação dos dados para modelagem, incluindo separação entre variáveis preditoras e alvo, criação de features derivadas e aplicação de pipelines para tratamento e transformação.

In [156]:
import pandas as pd

In [157]:
train_set = pd.read_csv("data/processed/train_set.csv")
test_set = pd.read_csv("data/processed/test_set.csv")

In [158]:
X_train = train_set.drop("median_house_value", axis=1)
y_train = train_set["median_house_value"].copy()

X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()



## 2. Identificação de Atributos Numéricos e Categóricos
Separação das colunas em numéricas e categóricas para tratamento diferenciado.

In [159]:
num_attribs = X_train.drop("ocean_proximity", axis=1).columns
cat_attribs = ["ocean_proximity"]

## 3. Criação de Transformer Personalizado
Definição de `CombinedAttributesAdder` para adicionar features derivadas:
- `rooms_per_household`
- `population_per_household`
- `bedrooms_per_room`

In [160]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

col_names = ["total_rooms", "total_bedrooms", "population", "households"]
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    X_train.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): 
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


## 4. Pipeline Numérico
Pipeline para variáveis numéricas, incluindo:
- Imputação de valores faltantes pela mediana
- Criação de novas features
- Escalonamento das variáveis

In [161]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

## 5. Pipeline Completo
Uso de `ColumnTransformer` para aplicar o pipeline numérico às variáveis numéricas e `OneHotEncoder` às variáveis categóricas.

In [162]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])


## 6. Transformação dos Conjuntos Treino e Teste
Aplicação do pipeline completo nos conjuntos de treino e teste, garantindo consistência entre eles.

In [163]:
X_train_set = full_pipeline.fit_transform(X_train)
X_test_set = full_pipeline.transform(X_test)

## 7. Salvamento dos Conjuntos Processados
Exportação das matrizes transformadas e dos alvos para uso na modelagem.

In [164]:

np.save("data/processed/X_train.npy", X_train_set)
y_train.to_csv("data/processed/y_train.csv", index=False)

np.save("data/processed/X_test.npy", X_test_set)
y_test.to_csv("data/processed/y_test.csv", index=False)
